<a href="https://colab.research.google.com/github/marcinmichalsmolen/adam/blob/main/kwiaty_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Import bibliotek
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

## Ładowanie obrazków i etykiet
(X_train, y_train), (X_test, y_test) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,
)

## Resizing obrazków
X_train = tf.image.resize(X_train, (150, 150))
X_test = tf.image.resize(X_test, (150, 150))

## Zmiana formatu etykiet
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

## Kopia danych na potrzeby kolejnych modeli
X_train2 = X_train
X_test2 = X_test
y_train2 = y_train
y_test2 = y_test


from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Ładowanie modelu VGG16 bez ostatnich warstw
base_model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
base_model.trainable = False ## Wagi nie będą trenowane

## Preprocessing input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model1 = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model1.summary()


from tensorflow.keras.callbacks import EarlyStopping

model1.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)


model1.fit(X_train, y_train, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


model1.evaluate(X_train, y_train)
model1.evaluate(X_test, y_test)